In [167]:
###importing packages
import newspaper
from newspaper import Article
from newspaper import fulltext
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import selenium 
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options

In [9]:
###establishing api key
NYT_KEY = %env NYT_KEY 
NYT_SECRET = %env NYT_SECRET

In [78]:
###creating function to pull NYT urls for climate articles
def api_call(i):
    params = {'api-key':NYT_KEY, 'q': ['climate change'], 'fq': ['article', 'The New York Times'], 
          'fl' : ['abstract', 'web_url', 'headline', 'pub_date', 'byline', 'keywords', 'word_count'],
          'begin_date': '20221101', 'end_date':'20231031', 'page': i}
    response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params = params)
    nyt_json = response.json()
    nyt_json_df = pd.DataFrame(nyt_json['response']['docs'])
    return nyt_json_df

In [154]:
###seeing how many hits there are
params = {'api-key':NYT_KEY, 'q': ['climate change'], 'fq': ['article', 'The New York Times'], 
          'fl' : ['abstract', 'web_url', 'headline', 'pub_date', 'byline', 'keywords', 'word_count'],
          'begin_date': '20221101', 'end_date':'20231031', 'page': i}
response = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json', params = params)
json = response.json()
json_df = pd.DataFrame(nyt_json['response']['docs'])

nyt_json['response']['meta']

{'hits': 3273, 'offset': 700, 'time': 49}

###My notes
pulled 100-200 pages of articles
need to pull 1-99 (100),201-301, 301-328

In [ ]:
dfs = []

In [164]:
###pulling articles for remaining pages of api call 
for i in range(100,201):
    time.sleep(12)
    dfs.append(api_call(i))

In [165]:
###checking length of dfs
len(dfs)

101

In [168]:
###combining df of each api pull
nyt_df = pd.concat(dfs, ignore_index = True)
nyt_df

,abstract,web_url,headline,keywords,pub_date,byline,word_count
0,The year 2022 was not great. But even in the m...,https://www.nytimes.com/2022/12/31/opinion/202...,{'main': 'Cheer Up! The World Is Better Off Th...,"[{'name': 'subject', 'value': 'Two Thousand Tw...",2022-12-31T14:00:07+0000,"{'original': 'By Nicholas Kristof', 'person': ...",984
1,"Plus, Nancy Pelosi leaves leadership and the w...",https://www.nytimes.com/2022/11/17/briefing/mh...,{'main': 'Your Friday Briefing: MH17 Convictio...,"[{'name': 'glocations', 'value': 'United State...",2022-11-17T21:31:55+0000,"{'original': 'By Amelia Nierenberg', 'person':...",1239
2,A video from the former president attacked ele...,https://www.nytimes.com/2023/07/20/us/politics...,{'main': 'Trump Seeks U.A.W.’s Support as the ...,"[{'name': 'subject', 'value': 'United States P...",2023-07-20T21:47:04+0000,"{'original': 'By Jonathan Weisman', 'person': ...",541
3,Pounding rainstorms following an epic drought ...,https://www.nytimes.com/2023/01/06/us/californ...,{'main': 'Wild Weather Swings Are Robbing Cali...,"[{'name': 'glocations', 'value': 'California',...",2023-01-06T10:00:27+0000,"{'original': 'By Shawn Hubler and Jill Cowan',...",1399
4,"In a win for the Biden administration, the Uni...",https://www.nytimes.com/2022/12/20/climate/pos...,"{'main': 'Your Mail Truck Is Going Electric', ...","[{'name': 'subject', 'value': 'Electric and Hy...",2022-12-20T21:22:05+0000,"{'original': 'By Brad Plumer', 'person': [{'fi...",843
...,...,...,...,...,...,...,...
1005,"Originally published in 1939, “The Hopkins Man...",https://www.nytimes.com/2023/01/02/books/revie...,"{'main': 'In the ‘Cozy Catastrophe’ Novel, the...","[{'name': 'subject', 'value': 'Books and Liter...",2023-01-02T10:00:12+0000,"{'original': 'By Alec Nevala-Lee', 'person': [...",1296
1006,A tornado killed a person west of New Orleans ...,https://www.nytimes.com/2022/12/13/us/texas-to...,{'main': 'Tornado Sweeps Through New Orleans a...,"[{'name': 'subject', 'value': 'Weather', 'rank...",2022-12-13T14:27:24+0000,"{'original': 'By Remy Tumin, McKenna Oxenden a...",937
1007,Voters’ broad discontent with disarray in Wash...,https://www.nytimes.com/2023/10/06/us/kevin-mc...,{'main': 'How Do Americans Feel About Politics...,"[{'name': 'subject', 'value': 'Polls and Publi...",2023-10-06T18:18:40+0000,"{'original': 'By Jack Healy, J. David Goodman,...",1447
1008,"Ford, General Motors and others are striking d...",https://www.nytimes.com/2023/07/02/business/li...,{'main': 'Lithium Scarcity Pushes Carmakers In...,"[{'name': 'subject', 'value': 'Automobiles', '...",2023-07-02T09:01:28+0000,{'original': 'By Clifford Krauss and Jack Ewin...,1417


In [170]:
###creating function to pull text from articles using urls pulled from nyt api
def article_download(pulled_url):
    article = Article(url = pulled_url, fetch_images = False, keep_article_html = True)
    article.download()
    article.parse()
    article.clean_top_node
    article_dict = {'text': article.text,
                   'html': article.html}
    return article_dict

In [171]:
###pulling out article text from every article url
text_df = []
for rows in range(len(nyt_df)):
    text = article_download(nyt_df['web_url'][rows])['text']
    text_df.append(text)

###adding column within nyt_df containing text
nyt_df['text'] = text_df

In [172]:
nyt_df

,abstract,web_url,headline,keywords,pub_date,byline,word_count,text
0,The year 2022 was not great. But even in the m...,https://www.nytimes.com/2022/12/31/opinion/202...,{'main': 'Cheer Up! The World Is Better Off Th...,"[{'name': 'subject', 'value': 'Two Thousand Tw...",2022-12-31T14:00:07+0000,"{'original': 'By Nicholas Kristof', 'person': ...",984,Enough with the doom and gloom! Our planet may...
1,"Plus, Nancy Pelosi leaves leadership and the w...",https://www.nytimes.com/2022/11/17/briefing/mh...,{'main': 'Your Friday Briefing: MH17 Convictio...,"[{'name': 'glocations', 'value': 'United State...",2022-11-17T21:31:55+0000,"{'original': 'By Amelia Nierenberg', 'person':...",1239,Three guilty of the downing of MH17\n\nA Dutch...
2,A video from the former president attacked ele...,https://www.nytimes.com/2023/07/20/us/politics...,{'main': 'Trump Seeks U.A.W.’s Support as the ...,"[{'name': 'subject', 'value': 'United States P...",2023-07-20T21:47:04+0000,"{'original': 'By Jonathan Weisman', 'person': ...",541,"Donald J. Trump, seeing an opening with organi..."
3,Pounding rainstorms following an epic drought ...,https://www.nytimes.com/2023/01/06/us/californ...,{'main': 'Wild Weather Swings Are Robbing Cali...,"[{'name': 'glocations', 'value': 'California',...",2023-01-06T10:00:27+0000,"{'original': 'By Shawn Hubler and Jill Cowan',...",1399,SACRAMENTO — The coast redwood crashed through...
4,"In a win for the Biden administration, the Uni...",https://www.nytimes.com/2022/12/20/climate/pos...,"{'main': 'Your Mail Truck Is Going Electric', ...","[{'name': 'subject', 'value': 'Electric and Hy...",2022-12-20T21:22:05+0000,"{'original': 'By Brad Plumer', 'person': [{'fi...",843,When the Postal Service made public a plan in ...
...,...,...,...,...,...,...,...,...
1005,"Originally published in 1939, “The Hopkins Man...",https://www.nytimes.com/2023/01/02/books/revie...,"{'main': 'In the ‘Cozy Catastrophe’ Novel, the...","[{'name': 'subject', 'value': 'Books and Liter...",2023-01-02T10:00:12+0000,"{'original': 'By Alec Nevala-Lee', 'person': [...",1296,“Journey’s End” was a surprise hit. The first ...
1006,A tornado killed a person west of New Orleans ...,https://www.nytimes.com/2022/12/13/us/texas-to...,{'main': 'Tornado Sweeps Through New Orleans a...,"[{'name': 'subject', 'value': 'Weather', 'rank...",2022-12-13T14:27:24+0000,"{'original': 'By Remy Tumin, McKenna Oxenden a...",937,Tornadoes continued to sweep across the South ...
1007,Voters’ broad discontent with disarray in Wash...,https://www.nytimes.com/2023/10/06/us/kevin-mc...,{'main': 'How Do Americans Feel About Politics...,"[{'name': 'subject', 'value': 'Polls and Publi...",2023-10-06T18:18:40+0000,"{'original': 'By Jack Healy, J. David Goodman,...",1447,Whitney Smith’s phone buzzed with a text from ...
1008,"Ford, General Motors and others are striking d...",https://www.nytimes.com/2023/07/02/business/li...,{'main': 'Lithium Scarcity Pushes Carmakers In...,"[{'name': 'subject', 'value': 'Automobiles', '...",2023-07-02T09:01:28+0000,{'original': 'By Clifford Krauss and Jack Ewin...,1417,Eager to avoid falling further behind Tesla an...


In [132]:
###if we want to pull full text using beautiful soup
soup_text = []
def SoupText(url):
    soup = BeautifulSoup(url, 'html.parser')
    text = soup.get_text()
    soup_text.append(text)

nyt_df['soup_text'] = soup_text





Why Smaller Storms Are Growing More Fearsome, More Often - The New York Times
  




















Skip to contentSkip to site indexSearch & Section NavigationSection NavigationSEARCHNew York Today’s PaperNew York|With Climate Change, Smaller Storms Are Growing More Fearsome, More Oftenhttps://www.nytimes.com/2023/09/30/nyregion/climate-change-flooding-storms.htmlShare full article384AdvertisementSKIP ADVERTISEMENTSupported bySKIP ADVERTISEMENTWith Climate Change, Smaller Storms Are Growing More Fearsome, More OftenThe Friday storm that produced vast flooding in New York City started out earlier in the week as an unremarkable — if unpredictable — weather system.Share full article384New Yorkers were taken by surprise Friday by the deluge of rain.Credit...Anna Watts for The New York TimesBy Hilary HowardSept. 30, 2023At first, it looked as if New York would simply be grazed by light rain on Friday.David Stark, a meteorologist with the National Oceanic and Atmospheric Administration

In [169]:
###trying other package https://github.com/susannapaoli/web-scraper-nyt
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
def get_body(url):
    driver.get(url)
    article_text = ''
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    paragraph = soup.find_all('p')
    for i in paragraph:
        a = i.get_text()
        if a != 'Advertisement' and a != 'Supported by' and a != 'Send any friend a story' and a != 'As a subscriber, you have 10 gift articles to give each month. Anyone can read what you share.' and not a.startswith("By"):
            article_text += a 
            article_text += " "
    time.sleep(8)
    return article_text

In [ ]:
nyt_df['article_body'] = nyt_df.apply(lambda x: get_body(x.web_url), axis=1) 

In [ ]:
driver.close()

In [178]:
###saving dataframe to csv file
nyt_df.to_csv("/Users/lisayokocarle/Desktop/Columbia/Fall2023/NLP/FinalProject/qmss-nlp-climate/data/article_data.csv")